In [44]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [45]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [46]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster','snow_m',
                          'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56', 'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']
    else:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster',
                          'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56', 'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+str(VER)+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]

In [47]:
def make_lag_roll(LAG_DAY,lag_df_new):
   
    lag_df = base_test[['id','d',TARGET]]
    
    lag_df=lag_df.sort_values(by=["d"])
  
    for i in range(0,len(LAG_DAY)):

        shift_day = LAG_DAY[i][0]
        roll_wind = LAG_DAY[i][1]
        col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
        lag_df[col_name] = (lag_df.groupby(['id'])[TARGET]).transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    lag_df_new=lag_df.drop(columns=["sales"])
    return lag_df_new

In [48]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    "lambda":0.1,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 



# lgb_params ={
#         "objective" : "tweedie",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
#         "metric": ["rmse"],
#         "nthread": -1,
#         "tweedie_variance_power":1.1,
#     'verbosity': 1,
# #     'num_iterations' : 1500,
#     'num_leaves': 128,
#     "min_data_in_leaf": 104,
#     }




# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [49]:
VER = 1                         # Our model version
SEED = 42                      # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1941               # End day of our train set, change this part for final
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28_v3.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_evaluation.csv')['store_id']#change this part for final
STORES_IDS = list(STORES_IDS.unique())

#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        ROLS_SPLIT.append([i,j])

In [51]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    print(features_columns)
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                            label=grid_df[train_mask][TARGET])
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                            label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+str(VER)+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    imp_type = "gain"
    features = estimator.feature_name()
    importances = estimator.feature_importance(imp_type)
    importance_df=pd.DataFrame(features,columns=['features'])
    importance_df['importances']=importances
    importance_df=importance_df.sort_values(by='importances', ascending=False)
    importance_df.to_csv(store_id+'_fe_imp_'+str(VER)+'.csv',index=False)
    del importance_df
    gc.collect()
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    #     !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()

Train CA_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7',

[100]	valid_0's rmse: 1.6165
[200]	valid_0's rmse: 1.58564
[300]	valid_0's rmse: 1.57228
[400]	valid_0's rmse: 1.56452
[500]	valid_0's rmse: 1.55813
[600]	valid_0's rmse: 1.55254
[700]	valid_0's rmse: 1.54732
[800]	valid_0's rmse: 1.54341
[900]	valid_0's rmse: 1.5391
[1000]	valid_0's rmse: 1.53434
[1100]	valid_0's rmse: 1.53027
[1200]	valid_0's rmse: 1.52712
[1300]	valid_0's rmse: 1.5237
[1400]	valid_0's rmse: 1.51953
Train TX_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y

[100]	valid_0's rmse: 1.92152
[200]	valid_0's rmse: 1.8751
[300]	valid_0's rmse: 1.8589
[400]	valid_0's rmse: 1.84767
[500]	valid_0's rmse: 1.83873
[600]	valid_0's rmse: 1.83075
[700]	valid_0's rmse: 1.82374
[800]	valid_0's rmse: 1.81763
[900]	valid_0's rmse: 1.81197
[1000]	valid_0's rmse: 1.80685
[1100]	valid_0's rmse: 1.80236
[1200]	valid_0's rmse: 1.79739
[1300]	valid_0's rmse: 1.79191
[1400]	valid_0's rmse: 1.7875


In [52]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day


for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    
    
    lag_df_new = pd.DataFrame()

    lag_df_new=make_lag_roll(ROLS_SPLIT,lag_df_new)


    grid_df = grid_df.merge(lag_df_new, on=['id','d'], how='left')


    for store_id in STORES_IDS:
        
        if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
                              'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept',
                              'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 
                              'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 
                              'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 
                              'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend',
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d',
                              'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 
                              'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 
                              'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36',
                              'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 
                              'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 
                              'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 
                              'rolling_mean_168', 'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14',
                              'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 'rolling_mean_tmp_7_7', 
                              'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56']
        else:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
                              'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept',
                              'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 
                              'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m',
                              'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 
                              'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend',
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d',
                              'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 
                              'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 
                              'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36',
                              'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 
                              'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 
                              'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 
                              'rolling_mean_168', 'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14',
                              'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 'rolling_mean_tmp_7_7', 
                              'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56']
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id

        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()

    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    
    del temp_df, lag_df_new

all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  3.32 min round |  3.32 min total |  40025.07 day sales |
Predict | Day: 2
##########  3.24 min round |  6.56 min total |  37458.79 day sales |
Predict | Day: 3
##########  3.25 min round |  9.80 min total |  36969.96 day sales |
Predict | Day: 4
##########  3.24 min round |  13.04 min total |  37205.43 day sales |
Predict | Day: 5
##########  3.23 min round |  16.28 min total |  42696.62 day sales |
Predict | Day: 6
##########  3.24 min round |  19.51 min total |  50600.06 day sales |
Predict | Day: 7
##########  3.24 min round |  22.76 min total |  51243.38 day sales |
Predict | Day: 8
##########  3.35 min round |  26.11 min total |  45702.07 day sales |
Predict | Day: 9
##########  3.47 min round |  29.59 min total |  38588.33 day sales |
Predict | Day: 10
##########  3.25 min round |  32.83 min total |  44032.25 day sales |
Predict | Day: 11
##########  3.23 min round |  36.06 min total |  45760.79 day sales |
Predict | Day: 12
##########  3.25 min round

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.867111,0.743878,0.737324,0.750862,1.059797,1.223126,1.071991,1.054523,0.827545,...,0.951151,1.398534,1.264924,0.949920,0.884117,0.816303,0.839176,1.112487,1.234140,1.047197
1,HOBBIES_1_002_CA_1_evaluation,0.253708,0.235781,0.217474,0.223412,0.241419,0.321174,0.364873,0.235215,0.200616,...,0.281103,0.346477,0.407846,0.232891,0.231682,0.264754,0.277113,0.333782,0.400082,0.403677
2,HOBBIES_1_003_CA_1_evaluation,0.529528,0.469513,0.487887,0.525088,0.710784,0.851356,0.872703,0.570320,0.493814,...,0.633409,0.718102,0.780227,0.476490,0.447064,0.441048,0.476135,0.712013,0.750810,0.745154
3,HOBBIES_1_004_CA_1_evaluation,1.441111,1.312879,1.249264,1.395185,1.927828,2.610987,2.814698,1.903631,1.538473,...,1.808288,2.572835,2.938365,1.560004,1.235587,1.306355,1.284150,1.851155,2.448656,2.729612
4,HOBBIES_1_005_CA_1_evaluation,1.036667,0.897841,0.831609,0.956129,1.146815,1.416807,1.421822,1.135640,0.920854,...,1.146352,1.476264,1.523069,1.089549,0.963214,1.006485,1.016344,1.285618,1.578678,1.379941


In [53]:
all_preds.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_evaluation,0.501285,0.441451,0.453954,0.499908,0.525509,0.504780,0.607582,0.525907,0.534854,...,0.613746,0.732856,0.815270,0.623706,0.560943,0.594622,0.491588,0.502761,0.525226,0.648685
30486,FOODS_3_824_WI_3_evaluation,0.261826,0.248651,0.245181,0.222874,0.245451,0.310390,0.322155,0.251221,0.242516,...,0.257237,0.433138,0.442154,0.308950,0.368986,0.373404,0.275135,0.274516,0.324278,0.364302
30487,FOODS_3_825_WI_3_evaluation,0.659162,0.539619,0.518667,0.466092,0.570930,0.614238,0.721637,0.656965,0.542690,...,0.797826,1.240518,1.364929,1.021106,1.128044,1.067333,0.690011,0.676586,0.704782,0.967966
30488,FOODS_3_826_WI_3_evaluation,1.132577,1.029730,1.019176,0.953032,1.141075,1.274362,1.317409,1.314202,1.142626,...,1.021361,1.406020,1.426979,1.181712,1.280234,1.271241,1.082259,1.224864,1.231444,1.277550
30489,FOODS_3_827_WI_3_evaluation,1.983914,1.775112,1.770023,1.808228,2.008200,2.223547,2.269385,1.854557,1.585777,...,1.616134,1.883817,1.886878,1.514798,1.750936,1.590838,1.535194,1.845653,1.839801,1.859959


In [54]:
all_preds.shape

(30490, 29)

In [55]:
all_preds.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.312728,1.228560,1.212527,1.220250,1.400348,1.659563,1.680662,1.498920,1.265606,1.444154,...,1.529111,1.924267,1.962608,1.521128,1.418491,1.487469,1.373017,1.545841,1.788693,1.656283
std,2.684826,2.508303,2.470913,2.501589,2.954999,3.488268,3.471243,3.079368,2.545706,2.854556,...,3.225877,4.090047,4.148652,3.080280,3.016216,3.140960,2.868527,3.335426,3.938500,3.642941
min,0.001752,0.013664,0.013017,0.014352,0.016618,0.016243,0.016417,0.014998,0.012613,0.015576,...,0.016879,0.017467,0.016811,0.013663,0.013356,0.013843,0.014628,0.017139,0.017124,0.016306
25%,0.264829,0.268687,0.267869,0.268367,0.299558,0.348233,0.355528,0.316996,0.277341,0.314779,...,0.316341,0.386074,0.392331,0.306826,0.284058,0.298224,0.288346,0.321110,0.359824,0.335300
50%,0.608482,0.580982,0.569106,0.569294,0.642259,0.761731,0.781921,0.695144,0.599359,0.677578,...,0.685529,0.862004,0.892476,0.703014,0.635077,0.664072,0.619982,0.694301,0.790549,0.749502
75%,1.329260,1.226819,1.219437,1.217568,1.386641,1.653845,1.687940,1.488297,1.261401,1.425602,...,1.492351,1.870914,1.921703,1.509795,1.379233,1.449535,1.333683,1.498532,1.713789,1.612908
max,103.778806,98.196701,98.038714,101.470954,137.047213,150.826170,142.712671,125.905916,106.296439,98.894248,...,133.372984,162.436418,169.315182,114.384694,108.417203,102.929456,106.248921,134.797246,171.981169,151.170475


In [56]:
# all the following is changed

In [57]:
# replace validation part
train_df = pd.read_csv('sales_train_evaluation.csv')
train_df=train_df[['id','d_1914','d_1915','d_1916','d_1917','d_1918','d_1919','d_1920','d_1921','d_1922','d_1923',
                  'd_1924','d_1925','d_1926','d_1927','d_1928','d_1929','d_1930','d_1931','d_1932','d_1933',
                  'd_1934','d_1935','d_1936','d_1937','d_1938','d_1939','d_1940','d_1941']]

In [58]:
train_df.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [59]:
submission = pd.read_csv('sample_submission.csv')

In [60]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
submission.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
train_df['id']=train_df['id'].str.replace('evaluation','validation')

In [63]:
train_df.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [64]:
train_df.columns=submission.columns

In [65]:
train_df.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [66]:
train_df.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_validation,0,0,0,2,2,0,0,0,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_validation,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_validation,0,0,1,1,0,2,1,1,0,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_validation,1,3,0,1,2,1,0,2,1,...,1,1,1,4,6,0,1,1,1,0
30489,FOODS_3_827_WI_3_validation,0,0,0,0,0,1,1,1,2,...,1,2,0,5,4,0,2,2,5,1


In [67]:
train_df.shape

(30490, 29)

In [68]:
submission.shape

(60980, 29)

In [69]:
submission = submission[['id']]
sub1 = submission.merge(train_df, on=['id'], how='left')

In [70]:
sub1.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [71]:
sub1.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60976,FOODS_3_824_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60977,FOODS_3_825_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60978,FOODS_3_826_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60979,FOODS_3_827_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
sub1=sub1[:30490]

In [73]:
sub1.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [74]:
sub1.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_validation,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,...,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
30486,FOODS_3_824_WI_3_validation,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
30487,FOODS_3_825_WI_3_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0
30488,FOODS_3_826_WI_3_validation,1.0,3.0,0.0,1.0,2.0,1.0,0.0,2.0,1.0,...,1.0,1.0,1.0,4.0,6.0,0.0,1.0,1.0,1.0,0.0
30489,FOODS_3_827_WI_3_validation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,...,1.0,2.0,0.0,5.0,4.0,0.0,2.0,2.0,5.0,1.0


In [75]:
sub2 = submission.merge(all_preds, on=['id'], how='left')

In [76]:
sub2.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
sub2.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.501285,0.441451,0.453954,0.499908,0.525509,0.504780,0.607582,0.525907,0.534854,...,0.613746,0.732856,0.815270,0.623706,0.560943,0.594622,0.491588,0.502761,0.525226,0.648685
60976,FOODS_3_824_WI_3_evaluation,0.261826,0.248651,0.245181,0.222874,0.245451,0.310390,0.322155,0.251221,0.242516,...,0.257237,0.433138,0.442154,0.308950,0.368986,0.373404,0.275135,0.274516,0.324278,0.364302
60977,FOODS_3_825_WI_3_evaluation,0.659162,0.539619,0.518667,0.466092,0.570930,0.614238,0.721637,0.656965,0.542690,...,0.797826,1.240518,1.364929,1.021106,1.128044,1.067333,0.690011,0.676586,0.704782,0.967966
60978,FOODS_3_826_WI_3_evaluation,1.132577,1.029730,1.019176,0.953032,1.141075,1.274362,1.317409,1.314202,1.142626,...,1.021361,1.406020,1.426979,1.181712,1.280234,1.271241,1.082259,1.224864,1.231444,1.277550
60979,FOODS_3_827_WI_3_evaluation,1.983914,1.775112,1.770023,1.808228,2.008200,2.223547,2.269385,1.854557,1.585777,...,1.616134,1.883817,1.886878,1.514798,1.750936,1.590838,1.535194,1.845653,1.839801,1.859959


In [78]:
sub2=sub2[30490:]

In [79]:
sub2.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0.867111,0.743878,0.737324,0.750862,1.059797,1.223126,1.071991,1.054523,0.827545,...,0.951151,1.398534,1.264924,0.949920,0.884117,0.816303,0.839176,1.112487,1.234140,1.047197
30491,HOBBIES_1_002_CA_1_evaluation,0.253708,0.235781,0.217474,0.223412,0.241419,0.321174,0.364873,0.235215,0.200616,...,0.281103,0.346477,0.407846,0.232891,0.231682,0.264754,0.277113,0.333782,0.400082,0.403677
30492,HOBBIES_1_003_CA_1_evaluation,0.529528,0.469513,0.487887,0.525088,0.710784,0.851356,0.872703,0.570320,0.493814,...,0.633409,0.718102,0.780227,0.476490,0.447064,0.441048,0.476135,0.712013,0.750810,0.745154
30493,HOBBIES_1_004_CA_1_evaluation,1.441111,1.312879,1.249264,1.395185,1.927828,2.610987,2.814698,1.903631,1.538473,...,1.808288,2.572835,2.938365,1.560004,1.235587,1.306355,1.284150,1.851155,2.448656,2.729612
30494,HOBBIES_1_005_CA_1_evaluation,1.036667,0.897841,0.831609,0.956129,1.146815,1.416807,1.421822,1.135640,0.920854,...,1.146352,1.476264,1.523069,1.089549,0.963214,1.006485,1.016344,1.285618,1.578678,1.379941


In [80]:
sub2.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.501285,0.441451,0.453954,0.499908,0.525509,0.504780,0.607582,0.525907,0.534854,...,0.613746,0.732856,0.815270,0.623706,0.560943,0.594622,0.491588,0.502761,0.525226,0.648685
60976,FOODS_3_824_WI_3_evaluation,0.261826,0.248651,0.245181,0.222874,0.245451,0.310390,0.322155,0.251221,0.242516,...,0.257237,0.433138,0.442154,0.308950,0.368986,0.373404,0.275135,0.274516,0.324278,0.364302
60977,FOODS_3_825_WI_3_evaluation,0.659162,0.539619,0.518667,0.466092,0.570930,0.614238,0.721637,0.656965,0.542690,...,0.797826,1.240518,1.364929,1.021106,1.128044,1.067333,0.690011,0.676586,0.704782,0.967966
60978,FOODS_3_826_WI_3_evaluation,1.132577,1.029730,1.019176,0.953032,1.141075,1.274362,1.317409,1.314202,1.142626,...,1.021361,1.406020,1.426979,1.181712,1.280234,1.271241,1.082259,1.224864,1.231444,1.277550
60979,FOODS_3_827_WI_3_evaluation,1.983914,1.775112,1.770023,1.808228,2.008200,2.223547,2.269385,1.854557,1.585777,...,1.616134,1.883817,1.886878,1.514798,1.750936,1.590838,1.535194,1.845653,1.839801,1.859959


In [81]:
final_sub=pd.concat([sub1,sub2],axis=0)

In [82]:
final_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [83]:
final_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.501285,0.441451,0.453954,0.499908,0.525509,0.504780,0.607582,0.525907,0.534854,...,0.613746,0.732856,0.815270,0.623706,0.560943,0.594622,0.491588,0.502761,0.525226,0.648685
60976,FOODS_3_824_WI_3_evaluation,0.261826,0.248651,0.245181,0.222874,0.245451,0.310390,0.322155,0.251221,0.242516,...,0.257237,0.433138,0.442154,0.308950,0.368986,0.373404,0.275135,0.274516,0.324278,0.364302
60977,FOODS_3_825_WI_3_evaluation,0.659162,0.539619,0.518667,0.466092,0.570930,0.614238,0.721637,0.656965,0.542690,...,0.797826,1.240518,1.364929,1.021106,1.128044,1.067333,0.690011,0.676586,0.704782,0.967966
60978,FOODS_3_826_WI_3_evaluation,1.132577,1.029730,1.019176,0.953032,1.141075,1.274362,1.317409,1.314202,1.142626,...,1.021361,1.406020,1.426979,1.181712,1.280234,1.271241,1.082259,1.224864,1.231444,1.277550
60979,FOODS_3_827_WI_3_evaluation,1.983914,1.775112,1.770023,1.808228,2.008200,2.223547,2.269385,1.854557,1.585777,...,1.616134,1.883817,1.886878,1.514798,1.750936,1.590838,1.535194,1.845653,1.839801,1.859959


In [84]:
final_sub.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,...,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000
mean,1.292523,1.196225,1.171121,1.179689,1.403618,1.656757,1.709993,1.457577,1.359500,1.371487,...,1.487301,1.852721,1.963938,1.455251,1.345142,1.352065,1.292658,1.470723,1.739181,1.719221
std,2.974124,2.742584,2.693906,2.700339,3.294300,3.794701,3.751586,3.269169,3.136229,3.092653,...,3.448315,4.262187,4.436203,3.199560,3.072539,3.074417,2.913631,3.426835,4.014834,3.977067
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062061,0.000000,0.000000,0.000000,...,0.000000,0.049625,0.098108,0.000000,0.000000,0.000000,0.000000,0.000000,0.027549,0.070063
50%,0.479894,0.444524,0.432311,0.430999,0.556312,0.743520,0.801136,0.605950,0.528651,0.554240,...,0.598741,0.870962,0.958744,0.614563,0.531889,0.534971,0.504158,0.608178,0.791461,0.780577
75%,1.249216,1.093864,1.066621,1.064552,1.402647,1.887757,2.000000,1.514387,1.274249,1.350331,...,1.515600,2.000000,2.000000,1.524843,1.305925,1.331398,1.225754,1.491095,2.000000,2.000000
max,133.000000,117.000000,113.000000,101.470954,196.000000,150.826170,142.712671,125.905916,143.000000,107.000000,...,143.000000,162.436418,187.000000,114.384694,108.417203,102.929456,106.248921,134.797246,171.981169,151.170475


In [85]:
final_sub.to_csv('lgb_bystore_final1.csv',index=False)